# Evaluate BBOB
## Helper Functions

In [1]:
from download_data import *

In [5]:
fn = "/home/benjamin/Dokumente/code/tmp/DAC-BO/exp_sweep/2022-09-24/16-55-11/wandb/latest-run/files/media/table/rollout_data_0_09e01fc14876733217c6.table.json"
table = load_wandb_table(fn)
table[table["bbob_dimension"] == 4]

,step,state,action,reward,instance,cost,configuration,initial_design,episode,policy_name,policy,seed,bbob_function,bbob_instance,bbob_dimension
100,0,[1.0],0,184671.715311,0,184671.715311,"[-3.6958975717425346, -3.8501164223998785, -0....","[[-1.6665607411414385, 0.08964152075350285, -2...",0,round robin,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",5,10,1,4
101,1,[0.9875],1,184671.715311,0,258913.634261,"[-0.39944759650458206, -0.009161928148659015, ...","[[-1.6665607411414385, 0.08964152075350285, -2...",0,round robin,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",5,10,1,4
102,2,[0.975],0,54090.747761,0,54090.747761,"[1.064375072090355, -0.2092120818804375, -2.62...","[[-1.6665607411414385, 0.08964152075350285, -2...",0,round robin,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",5,10,1,4
103,3,[0.9625],1,54090.747761,0,79246.653065,"[1.0378192789711544, -0.19247443797502317, -2....","[[-1.6665607411414385, 0.08964152075350285, -2...",0,round robin,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",5,10,1,4
104,4,[0.95],0,54090.747761,0,3247937.694369,"[-3.2445372842044726, -2.3329210120326334, -4....","[[-1.6665607411414385, 0.08964152075350285, -2...",0,round robin,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",5,10,1,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715,75,[0.0625],1,110.256173,0,2902.765559,"[-3.0331506715443926, -2.7542732816588855, 4.5...","[[-3.9931616839021444, -3.6365389078855515, -4...",0,round robin,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",6,11,1,4
716,76,[0.05],0,110.256173,0,125.027734,"[2.5614574002590604, -4.987678602463825, -4.02...","[[-3.9931616839021444, -3.6365389078855515, -4...",0,round robin,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",6,11,1,4
717,77,[0.0375],1,110.256173,0,562.041243,"[2.589562727970862, -4.994302418400474, -3.989...","[[-3.9931616839021444, -3.6365389078855515, -4...",0,round robin,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",6,11,1,4
718,78,[0.025],0,110.256173,0,5943.180573,"[-1.2156374665455032, -0.37334918514224924, 1....","[[-3.9931616839021444, -3.6365389078855515, -4...",0,round robin,"[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, ...",6,11,1,4


## Download Data from WandB

In [ ]:
runs_df = download(group="paris2")

## Convert into Plot Data

In [4]:
import pandas as pd
import ioh
from tqdm import tqdm

runs_fn = "/home/benjamin/Dokumente/code/tmp/DAC-BO/tmp/wandb_runs.pickle"
runs_df = pd.read_pickle(runs_fn)

reload = False
if reload:  # or not Path("../" + fn_rollout_data).is_file():
    data_list = []
    for index, run in tqdm(runs_df.iterrows(), total=len(runs_df)):
        if run["outdir"] is None:
            continue
        p = Path("..") / Path(run["outdir"])
        fn_cfg = p / fn_config
        fn_wbsum = p / fn_wbsummary
        fn_wbcfg = p / fn_wbconfig

        if fn_cfg.is_file():
            cfg = OmegaConf.load(fn_cfg)
            traincfg = recover_traincfg_from_wandb(fn_wbcfg)
            summary = lazy_json_load(fn_wbsum)

            path_to_table = fn_wbsum.parent / summary["rollout_data"]["path"]
            rollout_data = load_wandb_table(path_to_table)
            bbob_function_id = cfg["coco_instance"]["function"]
            bbob_dim = cfg["coco_instance"]["dimension"]
            bbob_instance = cfg["coco_instance"]["instance"]
            rollout_data["bbob_function"] = bbob_function_id
            rollout_data["bbob_dimension"] = bbob_dim
            rollout_data["bbob_instance"] = bbob_instance
            problem = ioh.get_problem(
                fid=bbob_function_id,
                instance=bbob_instance,
                dimension=bbob_dim,
                problem_type="BBOB",
            )
            optimum = problem.objective.y
            rollout_data["regret"] = rollout_data["reward"] - optimum

            data_list.append(rollout_data)

    df = pd.concat(data_list)
    df.reset_index(drop=True, inplace=True)
    df.to_csv(fn_rollout_data, index=False)
else:
    print(f"Load {fn_rollout_data}")
    df = pd.read_csv(fn_rollout_data)

# Scale regret
groups = df.groupby(by=["bbob_function", "bbob_dimension"])
new_df = []
for group_id, group_df in groups:
    group_df = scale(group_df)
    new_df.append(group_df)
df = pd.concat(new_df)

Load tmp/rollout_bbob.csv


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

data = df[df["bbob_dimension"] == 5]
groups = data.groupby(by=["bbob_function", "policy_name"])
for group_id, group_df in groups:
    # group_df = scale(group_df)
    ax = sns.histplot(data=group_df, x="regret_log_scaled")
    ax.set_title(group_id)
    plt.show()

## Export Final Regret Table

In [22]:
dimension = 5
table_fn = "tmp/ioh.csv"
translator = {
    "bbob_function": "function",
    "policy_name": "schedule",
    "seed": "seed",
    "regret": "final regret",
    "regret_log_scaled": "final scaled log regret",
}
data = df[df["bbob_dimension"] == dimension]
data = data[data["step"] == data["step"].max()]
# data = scale(data)
print(data.columns)
table = {v: data[k] for k, v in translator.items()}
table = pd.DataFrame(table)
table.reset_index(drop=True, inplace=True)
table.to_csv(table_fn)
print(table["final scaled log regret"].min(), table["final scaled log regret"].max())
print(len(table))


# import matplotlib.pyplot as plt
# import seaborn as sns
# ax = sns.violinplot(data=table, x="schedule", y="final scaled log regret", cut=0)
# ax = sns.stripplot(data=table, x="schedule", y="final scaled log regret", ax=ax, size=1, color="black")
# plt.show()

Index(['step', 'state', 'action', 'reward', 'instance', 'cost',
       'configuration', 'initial_design', 'episode', 'policy_name', 'policy',
       'seed', 'bbob_function', 'bbob_dimension', 'bbob_instance', 'regret',
       'regret_log_scaled'],
      dtype='object')
0.0 0.9807196400670181
10257


## Check for Missing Data

In [ ]:
# Check what data is missing
n_seeds_t = 60
n_fns_t = 24
n_sch_t = 7
n_dim_t = 2
print(df.columns)
group_keys = ["bbob_dimension", "bbob_function"]
groups = df.groupby(by=group_keys)
for group_id, group_df in groups:
    seeds = group_df["seed"].unique()
    seeds.sort()
    n_seeds = group_df["seed"].nunique()
    if n_seeds != n_seeds_t:

        print(group_id, n_seeds, seeds)

    n_policies = group_df["policy_name"].nunique()
    if n_policies != n_sch_t:
        policies = group_df["policy_name"].unique()
        policies.sort()
        print(group_id, n_policies, policies)
        # sgroups = group_df.groupby(by="policy_name")
        # for sid, sdf in sgroups:
        #     seeds = group_df["seed"].unique()
        #     seeds.sort()
        #     print(sid, seeds)

## Plot Functions Definitions

In [9]:
from plot_utils import *

In [ ]:
get_color_palette(df)

## Plot Regret over Steps

### Aggr. BBOB Functions

In [ ]:
df.columns
group_keys = ["bbob_dimension"]  # , "bbob_function"]
groups = df.groupby(by=group_keys)
for group_id, group_df in groups:
    if type(group_id) != list:
        group_id = [group_id]
    if group_id[0] != 5:
        continue
    sns.set_style("whitegrid")
    title = get_group_title(group_keys=group_keys, group_id=group_id)
    # plot_return(data=group_df, title=title)
    # plot_reward_over_steps(data=group_df, title=title)
    plot_final_regret(data=group_df, title=title, yname="regret_log_scaled")
    plot_regret_over_steps(data=group_df, title=title, yname="regret_log_scaled", errorbar="ci")

### Aggr. per Function Family

In [ ]:
dimension = 5
families = {
    "separable": [1, 2, 3, 4, 5],
    "low/moderate conditioning": [6, 7, 8, 9],
    "high conditioning, unimodel": [10, 11, 12, 13, 14],
    "multi-modal, global structure": [15, 16, 17, 18, 19],
    "multi-model, weak global structure": [20, 21, 22, 23, 24],
}
data = df[df["bbob_dimension"] == dimension]
group_keys = ["bbob_function"]
groups = df.groupby(by=group_keys)
new_dfs = {}
for family, members in families.items():
    new_group = []
    for group_id, group_df in groups:
        if group_id in members:
            new_group.append(group_df)
    new_df = pd.concat(new_group)
    new_dfs[family] = new_df
for family, group_df in new_dfs.items():
    title = f"{family}: {families[family]}"
    plot_final_regret(data=group_df, title=title, yname="regret_log_scaled")
    plot_regret_over_steps(data=group_df, title=title, yname="regret_log_scaled")

### Per BBOB Function


In [ ]:
sns.set_style("whitegrid")
sns.set_palette("colorblind")
df.columns
group_keys = ["bbob_dimension", "bbob_function"]
groups = df.groupby(by=group_keys)
for group_id, group_df in groups:
    if type(group_id) != tuple:
        group_id = [group_id]
    if group_id[0] != 5:
        continue
    title = get_group_title(group_keys=group_keys, group_id=group_id)
    # plot_return(data=group_df, title=title)
    # plot_reward_over_steps(data=group_df, title=title)
    plot_final_regret(data=group_df, title=title, yname="regret_log_scaled")
    plot_regret_over_steps(data=group_df, title=title, yname="regret_log_scaled", errorbar="ci")

## Plot All Samples for Each Schedule

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("whitegrid")
sns.set_palette("colorblind")
df.columns
data = df[df["bbob_dimension"] == 5]
# bbob_function = 19
# data = data[data["bbob_function"] == bbob_function]
group_keys = ["bbob_function"]
groups = data.groupby(by=group_keys)
for group_id, group_df in groups:
    if type(group_id) != tuple:
        group_id = [group_id]
    title = get_group_title(group_keys=group_keys, group_id=group_id)
    print(title)
    yname = "regret_log_scaled"
    errorbar = "sd"

    group_df = group_df.rename(columns={"policy_name": "schedule"})
    grid = sns.FacetGrid(data=group_df, col="schedule")
    grid.map_dataframe(sns.lineplot, x="step", y="regret_log_scaled", hue="seed", errorbar=errorbar)
    grid.set_xlabels("BO evaluations")
    grid.set_ylabels("log regret (scaled)")
    grid.set_titles(template="{col_name}")

    # Reference lines for percentages after which to switch
    n_steps = data["step"].max()
    x = [n_steps * k for k in [0.25, 0.5, 0.75]]
    for xi in x:
        grid.refline(x=xi, color="grey", alpha=0.25, ls="-")

    plt.show()
    basename = f"./tmp/figures/convergence_perschedule/regret_over_steps_hueseed_{title}"
    fig = grid.figure
    savefig(fig=fig, basename=basename)

    # continue

    # fig = plt.figure()
    # ax = fig.add_subplot(111)
    # n_steps = data["step"].max()
    # x = [n_steps * k for k in [0.25, 0.5, 0.75]]

    # ax = sns.lineplot(
    #     data=data,
    #     x="step",
    #     y=yname,
    #     hue="policy_name",
    #     ax=ax,
    #     palette=get_color_palette(data),
    #     errorbar=errorbar,
    # )
    # if not "log" in yname:
    #     ax.set_yscale("log")
    # # else:
    # #     ax.set_ylim(0, 1)
    # ymin, ymax = ax.get_ylim()
    # ax.vlines(x=x, ymin=ymin, ymax=ymax, color="grey", alpha=0.5)
    # xticks = np.linspace(0, n_steps + 1, 5)
    # xticks = [int(x) for x in xticks]
    # # ax.set_xticks(xticks)
    # title = title.replace("bbob_", "")
    # # ax.set_title(title)
    # ax.set_xlabel("evaluations")
    # ax.set_ylabel("log regret (scaled)")
    # ax.legend(title="schedule")
    # fig.set_tight_layout(True)
    # plt.show()
    # err = errorbar if errorbar == "_ci" else ""
    # basename = f"./tmp/figures/convergence/regret_over_steps{err}_{title}"
    # savefig(fig=fig, basename=basename)

In [ ]:
from rich import inspect

lines = ax.get_lines()
for line in lines:
    # inspect(line, methods=True)
    label = line.get_label()
    if label.startswith("explore"):
        print(label)
        # print(inspect(line, methods=True))
        print(line.get_xydata())
    # break

plt.show()

## Plot Search Space Coverage

Plot per BBOB function

'step', 'state', 'action', 'reward', 'instance', 'cost',
       'configuration', 'initial_design', 'episode', 'policy_name', 'policy',
       'seed', 'bbob_function', 'bbob_dimension', 'bbob_instance', 'regret'

In [ ]:
import ioh

bbob_dim = 2
df = df[df["bbob_dimension"] == bbob_dim]
group_keys = ["bbob_function"]  # , "seed"]
groups = data.groupby(by=group_keys)
for group_id, group_df in groups:
    print(group_id)
    n_ticks = 5

    if type(group_id) != list:
        group_id = [group_id]
    fid = group_id[0]

    # Draw contour plot for groundtruth
    problem = ioh.get_problem(fid=fid, instance=1, dimension=bbob_dim, problem_type="BBOB")
    lb = problem.constraint.lb
    ub = problem.constraint.ub
    n_levels = 20
    n_points = 1000
    X = np.linspace(lb[0], ub[0], n_points)
    Y = np.linspace(lb[1], ub[1], n_points)
    XX, YY = np.meshgrid(X, Y)
    # ZZ = np.zeros(XX.flatten().shape[0])
    ZZ = np.zeros_like(XX)
    for i in range(n_points):
        for j in range(n_points):
            x = XX[i, j]
            y = YY[i, j]
            z = problem([x, y])
            ZZ[i, j] = z
    # ZZ = ZZ.reshape(XX.shape)

    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.contourf(XX, YY, ZZ, levels=n_levels, cmap="viridis")

    # Add sampled points from different schedules
    X = group_df["configuration"].to_list()
    X = np.array([np.array(ast.literal_eval(x)) for x in X])

    names = group_df["policy_name"].reset_index(drop=True)
    series = [pd.Series(data=X[:, i], name=f"x{i}") for i in range(X.shape[-1])] + [names]
    sub_df = pd.concat(series, axis=1)

    ax = sns.scatterplot(data=sub_df, x="x0", y="x1", hue="policy_name", ax=ax)
    title = get_group_title(group_keys=group_keys, group_id=group_id)
    ax.set_title(title)

    xticks = np.linspace(lb[0], ub[0], n_ticks)
    yticks = np.linspace(lb[1], ub[1], n_ticks)
    ax.set_xticks(xticks)
    ax.set_yticks(yticks)

    # margin = 0.05
    # ax.set_xlim((1 - margin) * lb[0], (1 + margin) * ub[0])
    # ax.set_ylim((1 - margin) * lb[1], (1 + margin) * ub[1])

    fig.set_tight_layout(True)
    plt.show()

    # break

## Plot 2D Landscape of BBOB function

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

import ioh

function = 16
dimension = 2
n_ticks = 5

# Draw contour plot for groundtruth
problem = ioh.get_problem(fid=function, instance=1, dimension=dimension, problem_type="BBOB")
print(problem)
lb = problem.constraint.lb
ub = problem.constraint.ub
n_levels = 25
n_points = 2000
X = np.linspace(lb[0], ub[0], n_points)
Y = np.linspace(lb[1], ub[1], n_points)
XX, YY = np.meshgrid(X, Y)
# ZZ = np.zeros(XX.flatten().shape[0])
ZZ = np.zeros_like(XX)
for i in range(n_points):
    for j in range(n_points):
        x = XX[i, j]
        y = YY[i, j]
        z = problem([x, y])
        ZZ[i, j] = z
# ZZ = ZZ.reshape(XX.shape)

fig = plt.figure(figsize=(6, 6), dpi=300)
ax = fig.add_subplot(111)
ax.contour(XX, YY, ZZ, levels=n_levels, cmap="viridis")
ax.set_aspect("equal")

xticks = np.linspace(lb[0], ub[0], n_ticks)
yticks = np.linspace(lb[1], ub[1], n_ticks)
ax.set_xticks(xticks)
ax.set_yticks(yticks)

ax.set_xlabel("$x_0$")
ax.set_ylabel("$x_1$")

# margin = 0.05
# ax.set_xlim((1 - margin) * lb[0], (1 + margin) * ub[0])
# ax.set_ylim((1 - margin) * lb[1], (1 + margin) * ub[1])

fig.set_tight_layout(True)
plt.show()

# break